In [134]:
import sys
sys.path.append("../code/soundsep")

In [135]:
SPECTROGRAMS_PATH = 
WAVFILE_PATH = 
INTERVALS_PATH = 

In [136]:
import hdbscan
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Audio, HTML, clear_output
from ipywidgets import interact, interactive
from sklearn.decomposition import PCA
from soundsig.sound import plot_spectrogram, spectrogram
from soundsig.signal import bandpass_filter

from interfaces.audio import LazyMultiWavInterface
from plotting_utils import MultiSpecPlotter

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
audio_signal = LazyMultiWavInterface([WAVFILE_PATH])
spectrograms = np.load(SPECTROGRAMS_PATH)[()]
intervals = np.load(INTERVALS_PATH)[()]

In [138]:
fig = None
def view_spectrogram(interval_idx, window_width, dbnoise):
    global fig
    
    if fig is not None:
        plt.close(fig)
        
    t_start, t_stop = intervals[interval_idx]
    
    t_mid = (t_stop + t_start) / 2
    
    window_width /= 2
    slice_start = max(0.0, t_mid - window_width)
    slice_stop = min(len(audio_signal) / audio_signal.sampling_rate, t_mid + window_width)
    _, sig = audio_signal.time_slice(slice_start, slice_stop)
    sig -= np.mean(sig, axis=0)
    sig = bandpass_filter(sig.T, sample_rate=audio_signal.sampling_rate, low_freq=250, high_freq=10000).T
    sig = sig[:, 0]
    
    fig = plt.figure(figsize=(16, 6))
    ax = plt.gca()
    
    t_spec, f_spec, spec, _ = spectrogram(sig, audio_signal.sampling_rate, 1000, 50, min_freq=250, max_freq=10000, cmplx=False)
    plot_spectrogram(slice_start + t_spec, f_spec, spec, ax=ax, dBNoise=dbnoise)
    
    idx1 = np.searchsorted(intervals[:, 1], slice_start)
    idx2 = np.searchsorted(intervals[:, 0], slice_stop)
    
    for i in np.arange(idx1, idx2):
        if i == interval_idx:
            continue
        t1, t2 = intervals[i]
        print(t1, t2)
        plt.vlines([t1], *plt.ylim(), color="Black", linestyle="--")
        plt.vlines([t2], *plt.ylim(), color="Black", linestyle="-")

    plt.vlines([t_start, t_stop], *plt.ylim(), color="Red", linestyle="-")
    plt.xlim(slice_start, slice_stop)
    
    plt.show()
    
    snip_start = max(0.0, t_start - 0.1)
    snip_stop = min(len(audio_signal) / audio_signal.sampling_rate, t_stop + 0.1)
    _, sig2 = audio_signal.time_slice(snip_start, snip_stop)
    sig2 -= np.mean(sig2, axis=0)
    sig2 = bandpass_filter(sig2.T, sample_rate=audio_signal.sampling_rate, low_freq=250, high_freq=10000).T
    sig2 = sig2[:, 0]

    display(Audio(data=sig2, rate=audio_signal.sampling_rate))
    print("Play full audio")
    display(Audio(data=sig, rate=audio_signal.sampling_rate))

    return

In [139]:
index_picker = widgets.IntSlider(
    value=0,
    min=0,
    max=len(intervals) - 1,
    step=1,
    description='Index:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

index_input = widgets.IntText(
    value=0,
    description='Pick Index:',
    disabled=False
)

set_window_width = widgets.FloatSlider(
    value=2.0,
    min=1.0,
    max=5.0,
    step=0.5,
    description="Window:",
    orientation="horizontal",
    continuous_update=False,
    readout=True,
    readout_format="f"
)

set_dbnoise = widgets.IntSlider(
    value=40,
    min=20,
    max=80,
    step=1,
    description="dBNoise",
    orientation="horizontal",
    continuous_update=False,
    readout=True,
    readout_format="d"
)

widgets.jslink((index_picker, "value"), (index_input, "value"))

out_spectrogram = widgets.interactive_output(
    view_spectrogram,
    {
        "interval_idx": index_picker,
        "window_width": set_window_width,
        "dbnoise": set_dbnoise
    }
)

prev_button = widgets.Button(
    description="Previous",
)

next_button = widgets.Button(
    description="Next",
)

def increment(_):
    index_picker.value += 1

def decrement(_):
    index_picker.value -= 1
    
next_button.on_click(increment)
prev_button.on_click(decrement)

In [140]:
widgets.VBox([
    out_spectrogram,
    widgets.HBox([
        index_input,
        prev_button,
        index_picker,
        next_button
    ]),
    set_window_width,
    set_dbnoise
])
